<a href="https://colab.research.google.com/github/maciejskorski/distorted_clustering/blob/main/Clustering_DistortedMetric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary

This notebook demonstrates how to cluster under distorted metrics, like the huber-like loss functions. Such metrics have been recently shown to be statistically more robust and accuracte for various machine learning tasks (see the paper by Google https://arxiv.org/abs/1701.03077)

Unfortunately, fitting such models is harder than for KMeans, and is not supported in off-the-shell software, such as scikitlearn. 

This notebook offers a Tensorflow implementation along with examples on real-world data.

# Model

We use the Expectation-Maximization algorithm (EM). The idea is essentially as in the case of KMeans; however KMeans works under the euclidean distance and allows for solving the M step analytically. In our case we resort to generic (gradient-based) optimization.

In [1]:
### Tensorflow Model

%tensorflow_version 1.x
import tensorflow as tf

def logp_huber(scale=1):

  def logp_huber(x):
    return -scale**2*((1+tf.square(x/scale))**0.5-1)
  
  return logp_huber

def logp_euclidean():

  def logp_euclidean(x):
    return -tf.square(x)

  return logp_euclidean

def build_cluster_graph(data_shape,n_clusters,logp_func=logp_euclidean,init=None):

  ## model variables 

  n_rows,n_features = data_shape
  X_t = tf.placeholder(tf.float32,shape=[n_rows,n_features]) # data
  y_logp = tf.Variable(tf.log(tf.ones(shape=[n_rows,n_clusters])/n_features),trainable=False) # class predictions, updated by Bayes rules
  sigmas = tf.ones(shape=[n_features]) # standard deviations: here fixed
  if init is None:
    init = tf.random.normal(shape=[n_clusters,n_features])
  centers = tf.Variable(init,trainable=True) # centers: trainable

  ## data probability given class predictions and model params: log Pr[x|y,model]

  diff = tf.expand_dims(X_t,axis=1) - tf.expand_dims(centers,0) # [n_rows,n_clusters,n_features]
  X_logp = tf.reduce_sum(logp_func(diff),axis=2) # [n_rows,n_clusters]

  ## optimize params given class predictions: log Pr[x|y]

  loglike = tf.reduce_logsumexp(y_logp + X_logp,axis=1) # [n_rows]
  neg_loglike = -tf.reduce_mean(loglike) # []
  optimizer = tf.train.AdamOptimizer()
  mstep = optimizer.minimize(neg_loglike)

  ## optimize class predictions given params via Bayes: Pr[y|x] := Pr[x|y]Pr[y] / Pr[x] 

  y_logp_new = X_logp+y_logp - tf.reduce_logsumexp(X_logp+y_logp,axis=1,keepdims=True)
  estep = tf.assign(y_logp,y_logp_new)

  init_op = tf.global_variables_initializer()

  return X_t,y_logp,mstep,estep,neg_loglike

TensorFlow 1.x selected.


# Tests


We demonstrate improvements when using Huber loss for clustering, 
on some standard clustering benchmarks.

In [2]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import adjusted_rand_score

## User Knowledge Dataset


In [3]:
## download and prepare data

!mkdir /content/user_knowledge
!curl -o /content/user_knowledge/data.xls 'https://archive.ics.uci.edu/ml/machine-learning-databases/00257/Data_User_Modeling_Dataset_Hamdi%20Tolga%20KAHRAMAN.xls'
X=pd.read_excel('user_knowledge/data.xls',sheet_name=1)
X=X[X.columns[:6]]
print(X)
X=X.to_numpy()
X,y=X[:,:-1],X[:,-1]
X=X.astype('float')
encode_dict={v:k for k,v in enumerate(np.unique(y))}
y=np.array(list(encode_dict[i] for i in y))
X=(X-X.mean(0))/X.std(0)

mkdir: cannot create directory ‘/content/user_knowledge’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57856  100 57856    0     0   136k      0 --:--:-- --:--:-- --:--:--  136k
      STG   SCG   STR   LPR   PEG       UNS
0    0.00  0.00  0.00  0.00  0.00  very_low
1    0.08  0.08  0.10  0.24  0.90      High
2    0.06  0.06  0.05  0.25  0.33       Low
3    0.10  0.10  0.15  0.65  0.30    Middle
4    0.08  0.08  0.08  0.98  0.24       Low
..    ...   ...   ...   ...   ...       ...
253  0.61  0.78  0.69  0.92  0.58      High
254  0.78  0.61  0.71  0.19  0.60    Middle
255  0.54  0.82  0.71  0.29  0.77      High
256  0.50  0.75  0.81  0.61  0.26    Middle
257  0.66  0.90  0.76  0.87  0.74      High

[258 rows x 6 columns]


In [4]:
## cluster by KMeans

np.random.seed(123)
y_pred = KMeans(n_clusters=len(np.unique(y)),n_init=10).fit_predict(X)
adjusted_rand_score(y,y_pred)

0.16055981705877584

In [5]:
## cluster by Huber

outs = []

n_clusters=len(np.unique(y))
n_rows,n_features = X.shape

for _ in range(10):
  
  for logp_func in [logp_huber(0.25)]:

    tf.reset_default_graph()
    init = np.random.normal(size=(n_clusters,n_features)).astype(np.float32)
    X_t,y_logp,mstep,estep,loglike = build_cluster_graph((n_rows,n_features),n_clusters,logp_func=logp_func,init=init)
    init_op = tf.global_variables_initializer()

    ## train
    with tf.Session() as sess:
      feed_dict = {X_t:X}
      sess.run(init_op)
      # do iterations
      val = 0
      for i in range(50):
        # m step (many times to get close to maximum likelihood)
        val_m = 0
        for _ in range(1000):
          _,val_m_new = sess.run([mstep,loglike],feed_dict=feed_dict)
          if abs(val_m_new-val_m) < 1e-5:
            break
          else:
            val_m = val_m_new
        # e step (once as it is analytic)
        sess.run(estep,feed_dict=feed_dict)

      y_pred = sess.run(y_logp,feed_dict)

    score = adjusted_rand_score(y,y_pred.argmax(1))
    outs.append((logp_func.__name__,val_m,score))
    
outs = pd.DataFrame(outs,columns=['method','loglike','ARI'])
print(outs.sort_values('loglike')[:2].mean())
outs

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
loglike    0.552995
ARI        0.197560
dtype: float64


,method,loglike,ARI
0,logp_huber,0.561601,0.132474
1,logp_huber,0.552683,0.178530
2,logp_huber,0.555267,0.177655
3,logp_huber,0.553710,0.178274
4,logp_huber,0.554309,0.134360
5,logp_huber,0.556716,0.187721
6,logp_huber,0.556431,0.174161
7,logp_huber,0.563748,0.102345
8,logp_huber,0.553306,0.216591
9,logp_huber,0.565495,0.139099


## Mice Protein Nuclear Dataset

In [6]:
## download and prepare data

!mkdir /content/mice
!curl -o /content/mice/data.xls 'https://archive.ics.uci.edu/ml/machine-learning-databases/00342/Data_Cortex_Nuclear.xls'

X=pd.read_excel('mice/data.xls')
# do one-hot-encoding for categorical data
print(X.head())
for c in X.columns[X.dtypes=='object']:
  encode_dict = {v:k for k,v in enumerate(X[c].unique())}
  X[c] = X[c].apply(encode_dict.get)
# fill missing data
X = X.fillna(X.median())
X = X.to_numpy()
X,y = X[:,:-1],X[:,-1]
X = (X-X.mean(0))/X.std(0)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1590k  100 1590k    0     0  2005k      0 --:--:-- --:--:-- --:--:-- 2003k
  MouseID  DYRK1A_N   ITSN1_N    BDNF_N  ...  Genotype  Treatment  Behavior   class
0   309_1  0.503644  0.747193  0.430175  ...   Control  Memantine       C/S  c-CS-m
1   309_2  0.514617  0.689064  0.411770  ...   Control  Memantine       C/S  c-CS-m
2   309_3  0.509183  0.730247  0.418309  ...   Control  Memantine       C/S  c-CS-m
3   309_4  0.442107  0.617076  0.358626  ...   Control  Memantine       C/S  c-CS-m
4   309_5  0.434940  0.617430  0.358802  ...   Control  Memantine       C/S  c-CS-m

[5 rows x 82 columns]


In [7]:
## cluster by KMeans

np.random.seed(123)
y_pred = KMeans(n_clusters=len(np.unique(y)),n_init=10).fit_predict(X)
adjusted_rand_score(y,y_pred)

0.1437098069247538

In [8]:
## cluster by Huber

outs = []

n_clusters=len(np.unique(y))
n_rows,n_features = X.shape

for _ in range(10):
  
  for logp_func in [logp_huber(0.25)]:

    tf.reset_default_graph()
    init = np.random.normal(size=(n_clusters,n_features)).astype(np.float32)
    X_t,y_logp,mstep,estep,loglike = build_cluster_graph((n_rows,n_features),n_clusters,logp_func=logp_func,init=init)
    init_op = tf.global_variables_initializer()

    ## train
    with tf.Session() as sess:
      feed_dict = {X_t:X}
      sess.run(init_op)
      # do iterations
      val = 0
      for i in range(50):
        # m step (many times to get close to maximum likelihood)
        val_m = 0
        for _ in range(1000):
          _,val_m_new = sess.run([mstep,loglike],feed_dict=feed_dict)
          if abs(val_m_new-val_m) < 1e-5:
            break
          else:
            val_m = val_m_new
        # e step (once as it is analytic)
        sess.run(estep,feed_dict=feed_dict)

      y_pred = sess.run(y_logp,feed_dict)

    score = adjusted_rand_score(y,y_pred.argmax(1))
    outs.append((logp_func.__name__,val_m,score))
    
outs = pd.DataFrame(outs,columns=['method','loglike','ARI'])
print(outs.sort_values('loglike')[:2].mean())
outs

loglike    8.102389
ARI        0.192036
dtype: float64


,method,loglike,ARI
0,logp_huber,8.148839,0.196426
1,logp_huber,8.078822,0.198950
2,logp_huber,8.155883,0.180153
3,logp_huber,8.132080,0.219714
4,logp_huber,8.154950,0.187463
5,logp_huber,8.228038,0.165376
6,logp_huber,8.196463,0.155176
7,logp_huber,8.125957,0.185122
8,logp_huber,8.261502,0.158310
9,logp_huber,8.127641,0.180094
